# Simple Neural Network in Pytorch

1. Fully connected neural network

In [1]:
#Imports 
import torch #entire lib
import torch.nn as nn #all neural network modules
import torch.optim as optim #optimizers adam,sgd etc
import torch.nn.functional as F #activation functions
from torch.utils.data import DataLoader #data loader
import torchvision.datasets as datasets # inbuilt datsets
import torchvision.transforms as transforms #tranformation on datasets

# Fully Connected Network

In [2]:
class NN(nn.Module):
    # call the super init method i.e for Module class
    # our input size = 784 (28x28) images (MNIST DATASET)
     
    def __init__(self,input_size,num_classes):
        super(NN,self).__init__()
        self.fc1 = nn.Linear(input_size,50)
        self.fc2 = nn.Linear(50,num_classes)
    
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x        

# Set Device 

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameter Optimization

In [4]:
input_size = 784
num_classes = 10 
learning_rate = 0.001
batch_size = 64
num_epochs = 1 

# Load Dataset

In [5]:
train_dataset = datasets.MNIST(root='dataset/',train=True,transform=transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataset = datasets.MNIST(root='dataset/',train=False,transform=transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

Extracting dataset/MNIST\raw\train-images-idx3-ubyte.gz to dataset/MNIST\raw


Extracting dataset/MNIST\raw\train-labels-idx1-ubyte.gz to dataset/MNIST\raw


Extracting dataset/MNIST\raw\t10k-images-idx3-ubyte.gz to dataset/MNIST\raw


Extracting dataset/MNIST\raw\t10k-labels-idx1-ubyte.gz to dataset/MNIST\raw
Processing...


..\torch\csrc\utils\tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


Done!


# Initialize Network

In [6]:
model = NN(input_size = input_size,num_classes=num_classes).to(device)

# Loss and Optimizer

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

# Train Network

In [12]:
for epoch in range(num_epochs):
    for batch_idx,(data,targets) in enumerate(train_loader):
        # data to cuda
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # correct shape of data
        data = data.reshape(data.shape[0],-1)
        
        #forward
        scores = model(data)
        loss = criterion(scores,targets)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        #gradient descent
        optimizer.step()
        
print(data.shape)

torch.Size([32, 784])


In [15]:
def check_accuracy(loader,model):
    if loader.dataset.train:
        print("Accuracy on Train")
    else:
        print("Accuracy on Test")
    
    num_correct = 0 
    num_samples = 0 
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0],-1)
            
            
            scores = model(x)
            # 64x10
            _,preds = scores.max(1)
            num_correct+= (preds==y).sum()
            num_samples+= preds.size(0)
        
        print(f" got accuracy {float(num_correct)/float(num_samples)*100:.2f}")
    
    model.train()


check_accuracy(train_loader,model)
check_accuracy(test_loader,model)

Accuracy on Train
 got accuracy 93.36
Accuracy on Test
 got accuracy 93.38
